图像分类问题

1. 图像的张量表示
图像长和宽，每个像素0~255的值，灰度图像只有一个通道，可以看成一个矩阵；彩色图像rgb三个通道，3个这样矩阵叠在一起的张量。

2. 卷积，滤波器
卷积网络借鉴了猫的视觉神经研究成果，某类细胞只对特定图像模式有反应，比如：图像种的水平纹理，垂直纹理，倾斜纹理，弧形纹理，颜色组合等，一幅图像经过这些简单模式识别后，再反复组合这种模式，就能构造出更复杂的模式。世界的表征是丰富多彩的，但基础组件是极其简单的，通过对基础组件的排列组合，构成了眼前的世界。 配图：基础组件+组合+最终人脸识别 那个直观示例图。

想象一下，有个3*3像素大小的小窗口，在图片上，从左到右，再从上到下，扫描图片种的每个地方，发现有直线纹理，就输出一个提示，这里有直线，如果没有就过了。这个小窗口叫滤波器, 也叫卷积核。 这个卷积核怎么跟图像扫描到的区域发生作用呢？

扫描到的图像部分也是3*3大小的窗口，这个区域的像素值x00,x01,x02,x10,x11,x12,x20,x21,x22, 卷积核w00,w01,w02,w10,w11,w12,w20,w21,w22. 从前面的全连接网络， w00*x00+w01*x01+...+w22*x22+bias=output, 这个就是加权和。w的值可以一开始随机初始化，通过学习得到具体的数值。

卷积核共享，局部连接，和全连接比，减少参数量。

3. 激活函数
sigmod(0~1.0),层数过多，导致梯度消失，RelU(0~∞)

4. 池化
相邻区域，更明显的特征是什么，平均池化，最大池化


3. 卷积网络要素，卷积，池化
4. LeNet
5. 其他卷积网络架构思想
6. 迁移学习

In [5]:
#Defining the convolutional neural network
class LeNet5(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet5, self).__init__()
        # 卷积 -> 激活 -> 池化，卷积三个基础操作
        # 卷积 -> 批归一化 -> 激活 -> 池化  #and drop out ？
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0), #1-通道，6-卷积核 5*5卷积核大小，步长1，填充0
            nn.BatchNorm2d(6), #批归一化
            nn.ReLU(), #激活，lenet为sigmoid，AlexNet开始relu
            nn.MaxPool2d(kernel_size = 2, stride = 2)) #最大池化
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        
#         self.fc = nn.Linear(400, 120)
#         self.relu = nn.ReLU()
#         self.fc1 = nn.Linear(120, 84)
#         self.relu1 = nn.ReLU()
#         self.fc2 = nn.Linear(84, num_classes)
        
        self.classifier = nn.Sequential(
             nn.Dropout(), #dropout,AlexNet引入
             nn.Linear(400, 120),
             nn.ReLU(),
             nn.Dropout(),
             nn.Linear(120, 84),
             nn.ReLU(),
             nn.Linear(84,num_classes)
         )
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        
#         out = self.fc(out)
#         out = self.relu(out)
#         out = self.fc1(out)
#         out = self.relu1(out)
#         out = self.fc2(out)
        
        out = self.classifier(out)
        
        return out

model = LeNet5(10)

1. LeNet: 卷积+激活+池化
2. AlexNet: 图像数据增广，Sigmoid->ReLu, 批归一化Batch Normalization，GPU训练
3. VGG
4. GoogLeNet, Inception模块
5. ResNet,

模型压缩部署,解决移动端部署问题，MobileNet

https://zhuanlan.zhihu.com/p/50045821

Depthwise Separable Convolution，将跨通道的3*3卷积 -> 单通道的3*3卷积+跨通道的1*1卷积

ReLU的在通道数较少的Feature Map上有非常严重信息损失问题，由此引入了Linear Bottlenecks和Inverted Residual。

Depthwise卷积

Pointwise卷积用于特征合并以及升维或者降维。很自然的我们可以想到使用 1*1 卷积来完成这个功能。